# Staging Geo-coding test data

ref [GPC:ticket:140][140]

[140]: https://informatics.gpcnetwork.org/trac/Project/ticket/140

Subject: example data dictionary/codebook and data file  
From: David Van Riper [mailto:vanriper@umn.edu]  
Sent: Tuesday, July 07, 2015 1:34 PM  
To: Mei Liu  
 
fwd by Mei Tuesday, July 07, 2015 1:54 PM

In [63]:
import pandas as pd
import numpy as np
dict(pandas=pd.__version__,
     numpy=pd.__version__)

{'numpy': u'0.19.0', 'pandas': u'0.19.0'}

## Field Definitions

In [64]:
def mn_census_data():
    from pathlib import Path
    return Path('/d1/geo-census/mn-census-data')

bk = pd.read_csv((mn_census_data() / 'acs_20135a/index_of_data_fields__acs_20135a.csv').open('rb'))
bk[['data_type', 'variable_code', 'variable_label']].head()

,data_type,variable_code,variable_label
0,t,FILEID,Always equal to ACS Summary File identification
1,t,STUSAB,State Postal Abbreviation
2,t,SUMLEVEL,Summary Level
3,t,COMPONENT,Geographic Component
4,n,LOGRECNO,Logical Record Number


In [65]:
bk.iloc[149]

data_type                                                                 n
dataset_code                                                2009_2013_ACS5a
table_source_code                                                    B02012
table_label               Native Hawaiian and Other Pacific Islander Alo...
table_universe            Native Hawaiian and Other Pacific Islander alo...
table_sequence                                                           21
variable_sequence                                                         1
variable_code                                                        UEW001
variable_label                                                        Total
start_column                                                           2133
width                                                                     9
implied_decimal_places                                                    0
multiplier                                                                1
appears_in_e

In [66]:
bk.columns

Index([u'data_type', u'dataset_code', u'table_source_code', u'table_label',
       u'table_universe', u'table_sequence', u'variable_sequence',
       u'variable_code', u'variable_label', u'start_column', u'width',
       u'implied_decimal_places', u'multiplier', u'appears_in_extracts'],
      dtype='object')

In [67]:
v = bk[bk.variable_code == 'UHD001'].iloc[0]
# v = bk[bk.variable_code == 'FILEID'].iloc[0]
v

data_type                                                                 n
dataset_code                                                2009_2013_ACS5a
table_source_code                                                    B19013
table_label               Median Household Income in the Past 12 Months ...
table_universe                                                   Households
table_sequence                                                          110
variable_sequence                                                         1
variable_code                                                        UHD001
variable_label            Median household income in the past 12 months ...
start_column                                                          16038
width                                                                     9
implied_decimal_places                                                    0
multiplier                                                                1
appears_in_e

In [5]:
# bk[bk.variable_label.str.contains('ducation')][['data_type', 'table_label', 'variable_code', 'variable_label', 'table_universe', u'table_sequence', u'variable_sequence']]

In [6]:
# bk[['table_source_code', 'table_label']].drop_duplicates().reset_index()

In [103]:
from textwrap import dedent

def column_def(field):
    dty = ('INTEGER' if field.data_type == 'n'  # TODO: implied_decimal_places, multiplier
           else 'VARCHAR2(%d)' % field.width)
    return '  {name} {dty}'.format(name=field.variable_code, dty=dty)

def field_spec(field):
    return '      %s position (%d-%d) char(%d)%s' % (
        field.variable_code,
        field.start_column, field.width,
        field.width,
        (" NULLIF %s = '.'" % field.variable_code
         if field.data_type == 'n' else ''))

def table_def(name, location, fields,
              data_dir='geo_census_stage',
              tools_dir='staging_tools'):
    if len(fields) > 1000:
        raise ValueError('ORA-01792: maximum number of columns in a table or view is 1000')
    coldefs = ',\n'.join(column_def(f)
                         for (_, f) in fields.iterrows())
    field_list = ',\n'.join(field_spec(f)
                            for (_, f) in fields.iterrows())
    return dedent('''\
    create table {name} (
    {coldefs}
    ) organization external (
      type oracle_loader
      default directory {data_dir}
      access parameters (
        records delimited by newline
        preprocessor {tools_dir}:'zcat.sh'
        fields lrtrim
        (
    {field_list}
        )
      )
      location ('{location}')
    )
    ''').format(
        name=name, coldefs=coldefs, field_list=field_list, location=location,
        data_dir=data_dir, tools_dir=tools_dir)

print table_def('acs_zcta_200', 'ge.00_file.dat.gz', bk[:5])

create table acs_zcta_200 (
  FILEID VARCHAR2(6),
  STUSAB VARCHAR2(2),
  SUMLEVEL VARCHAR2(3),
  COMPONENT VARCHAR2(2),
  LOGRECNO INTEGER
) organization external (
  type oracle_loader
  default directory geo_census_stage
  access parameters (
    records delimited by newline
    preprocessor staging_tools:'zcat.sh'
    fields lrtrim
    (
      FILEID position (1-6) char(6),
      STUSAB position (7-2) char(2),
      SUMLEVEL position (9-3) char(3),
      COMPONENT position (12-2) char(2),
      LOGRECNO position (14-7) char(7) NULLIF LOGRECNO = '.'
    )
  )
  location ('ge.00_file.dat.gz')
)



In [125]:
def field_groups(all_fields,
                 last_key='NAME', max_columns=1000):
    n_keys = all_fields[all_fields.variable_code == last_key].index.values[0]
    group = all_fields[:n_keys]
    g_ix = 0
    for table_code in all_fields.table_source_code[n_keys + 1:].unique():
        # print '========', table_code
        table_fields = all_fields[all_fields.table_source_code == table_code]
        # print table_fields.index
        if len(group) + len(table_fields) > max_columns:
            yield g_ix, group
            g_ix += 1
            group = all_fields[:n_keys]
        group = group.append(table_fields)
    if len(group) > n_keys:
        yield g_ix, group

[(ix, len(g)) for (ix, g) in field_groups(bk, max_columns=768)]

[(0, 761), (1, 740), (2, 745), (3, 762), (4, 742), (5, 427)]

In [126]:
with open('geo_acs_tables.sql', 'wb') as out:
    for g_ix, grp in field_groups(bk, max_columns=768):
        out.write(table_def('acs_zcta_%d' % g_ix, 'ge.00_file.dat.gz', grp))
        out.write(';\n\n')

## Income Field

Desired fields, from #140:
>  - Income, education, likelihood of employment, poverty status, owner-occupied house value, health insurance coverage, etc.

Let's start with just income by zip.

In [80]:
# vars = bk[bk.variable_code.isin(['ZCTA5', 'UHD001', 'UG4011', 'UGS001', 'UGS012', 'UGS017'])]
income_vars = bk[bk.variable_code.isin(['ZCTA5', 'UHD001', 'UEI001', 'NAME'])]

income_vars[['data_type', 'table_label', 'table_universe', 'variable_code', 'variable_label']]

,data_type,table_label,table_universe,variable_code,variable_label
37,t,NaN,NaN,ZCTA5,5-digit ZIP Code Tabulation Area
49,t,NaN,NaN,NAME,Area Name
113,n,Median Age by Sex (American Indian and Alaska ...,People who are American Indian and Alaska Nati...,UEI001,Median age: Total
1688,n,Median Household Income in the Past 12 Months ...,Households,UHD001,Median household income in the past 12 months ...


In [128]:
income_vars.variable_label.iloc[3]

'Median household income in the past 12 months (in 2013 inflation-adjusted dollars)'

In [8]:
# Education
# ed = bk[bk.table_source_code.isin(['B15003'])]
# ed[['data_type', 'table_label', 'table_universe', 'variable_code', 'variable_label']]

## Data Files

Data files are provided at various resolutions including state, county, zip code, all the way to the census block group:

In [9]:
!ls -s /d1/geo-census/mn-census-data/acs_20135a/*/*file.dat.gz | sort -n

   372 /d1/geo-census/mn-census-data/acs_20135a/metdiv_314/ge.00_file.dat.gz
   652 /d1/geo-census/mn-census-data/acs_20135a/state_040/ge.00_file.dat.gz
  8448 /d1/geo-census/mn-census-data/acs_20135a/cbsa_310/ge.00_file.dat.gz
 24176 /d1/geo-census/mn-census-data/acs_20135a/urb_area_400/ge.00_file.dat.gz
 25144 /d1/geo-census/mn-census-data/acs_20135a/county_050/ge.00_file.dat.gz
169080 /d1/geo-census/mn-census-data/acs_20135a/cty_sub_060/ge.00_file.dat.gz
174060 /d1/geo-census/mn-census-data/acs_20135a/zcta_860/ge.00_file.dat.gz
280628 /d1/geo-census/mn-census-data/acs_20135a/place_070/ge.00_file.dat.gz
913112 /d1/geo-census/mn-census-data/acs_20135a/blck_grp_150/ge.00_file.dat.gz


We can decompress and pick out the first line...

In [69]:
import gzip

# Zip code level data
zcta_860 = mn_census_data() / 'acs_20135a/zcta_860/ge.00_file.dat.gz'

line0 = gzip.GzipFile(zcta_860.name,
                      fileobj=zcta_860.open('rb')).readline()
line0[:40]

'ACSSF PR860000007371                    '

... and then pick out the median household income field (UHD001):

In [70]:
line0[v.start_column - 1:v.start_column - 1 + v.width]

'    12041'

Recall that's a numeric (n) field. Let's parse a dictionary record from a line and a selection of fields:

In [71]:
SUPPRESSED = '.'  # per UMN folks

def parse_n(s, implied_decimal_places):
    return (float(s) / (10 ** implied_decimal_places)
            if s.strip() != SUPPRESSED
            else None)

def parse_field(s, f):
    return parse_n(s, f.implied_decimal_places) if f.data_type == 'n' else s

def parse_record(line, fields):
    return dict((f.variable_code,
                 parse_field(line[f.start_column - 1:f.start_column - 1 + f.width], f))
                for _, f in fields.iterrows())

parse_record(line0, bk[bk.variable_code.isin(['STUSAB', 'ZCTA5', 'UHD001'])])

{'STUSAB': 'PR', 'UHD001': 12041.0, 'ZCTA5': '00601'}

Now we can parse a record from each line in a data file and make a data frame:

In [72]:
def load_lines(lines, fields):
    return pd.DataFrame([
            parse_record(line, fields)
            for (lineno, line) in enumerate(lines)
        ])



income_by_zip = load_lines(gzip.GzipFile(zcta_860.name, fileobj=zcta_860.open('rb')), income_vars).set_index('ZCTA5')
income_by_zip.head()

,UEI001,UHD001
ZCTA5,,
00601,NaN,12041.0
00602,NaN,15663.0
00603,NaN,15485.0
00606,NaN,15019.0
00610,NaN,16707.0


In [74]:
income_vars.variable_label.iloc[1]

'Median age: Total'

In [16]:
from io import StringIO

patient_dimension = pd.read_csv(StringIO(u"""
patient_num,zip_code
1,64108
2,90210
""".strip()), index_col='patient_num', dtype=dict(zip_code='object'))
patient_dimension

,zip_code
patient_num,
1,64108
2,90210


In [17]:
patient_dimension.merge(income_by_zip, how='left', left_on='zip_code', right_index=True)

,zip_code,UHD001
patient_num,,
1,64108,35521.0
2,90210,132254.0


In [ ]:
http://www.oracle.com/technetwork/issue-archive/2011/11-mar/o21nanda-312277.html